In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d columbine/imdb-dataset-sentiment-analysis-in-csv-format

imdb-dataset-sentiment-analysis-in-csv-format.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-sentiment-analysis-in-csv-format.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/Train.csv")

In [ ]:
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
df.sample(5)

,text,label
28703,A bloody maniac with cannibalistic tendencies ...,0
16084,I've read a number of reviews on this film and...,1
9917,The famous French detective Henri Cassin takes...,1
35346,"I am very sorry to say this but ""Thunderbirds""...",0
36931,"Steven buddy, you remember when you said this:...",0


In [ ]:
df['text'][0]

'I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'

In [ ]:
df.shape

(40000, 2)

In [ ]:
df['label'].value_counts()

0    20019
1    19981
Name: label, dtype: int64

In [ ]:
import nltk

nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
pattern = re.compile('<.*?>')

def transform(text):
  
  text = (re.sub(pattern,'',text)).replace('\'','')
  text = text.lower()
  text = nltk.word_tokenize(text)
  #text = text.split()
  y=[]
  for i in text:
    if i.isalnum():
      y.append(i)

  text = y[:]
  y.clear()

  for i in text:
    if  i not in stopwords.words("english") and i not in string.punctuation:
      y.append(i)

  text = y[:]
  y.clear()

  for i in text:
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
df.duplicated().sum()

277

In [ ]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
 df = df.drop_duplicates(keep='first')

 df = df.sample(20000)

In [ ]:
df['label'].value_counts()

0    10025
1     9975
Name: label, dtype: int64

In [ ]:
df['text'] = df['text'].apply(transform)

In [ ]:
df.head()

,text,label
6413,go way back page ten review section work way b...,1
19535,go admit saw first episod show time might cons...,0
37394,film origin releas christma 1940 long thought ...,0
8775,libbi talk desmond flashback anyon confus past...,1
22305,review sept 12 2006 2nd screen paramount 1 the...,1


In [ ]:
spam_corpus=[]
for msg in df[df['label']==0]['text'].tolist():
  for word in msg.split():
    spam_corpus.append(word)

In [ ]:
from collections import Counter
Counter(spam_corpus).most_common(50)

[('movi', 22787),
 ('film', 17585),
 ('one', 10332),
 ('like', 9912),
 ('make', 6268),
 ('even', 6194),
 ('get', 6116),
 ('time', 5927),
 ('good', 5886),
 ('watch', 5839),
 ('charact', 5697),
 ('bad', 5583),
 ('would', 5502),
 ('see', 5151),
 ('realli', 4836),
 ('look', 4600),
 ('scene', 4577),
 ('stori', 4272),
 ('act', 4131),
 ('much', 4056),
 ('dont', 4015),
 ('go', 3890),
 ('thing', 3775),
 ('peopl', 3728),
 ('end', 3714),
 ('could', 3641),
 ('think', 3596),
 ('plot', 3391),
 ('made', 3351),
 ('show', 3338),
 ('say', 3318),
 ('seem', 3306),
 ('first', 3253),
 ('way', 3228),
 ('know', 3167),
 ('well', 3163),
 ('tri', 3027),
 ('play', 2908),
 ('also', 2850),
 ('want', 2835),
 ('actor', 2816),
 ('ever', 2661),
 ('never', 2652),
 ('come', 2610),
 ('better', 2605),
 ('seen', 2574),
 ('take', 2483),
 ('actual', 2430),
 ('work', 2380),
 ('two', 2374)]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

cv= CountVectorizer()
tf= TfidfVectorizer(max_features=20000)
x = tf.fit_transform(df['text']).toarray()

In [ ]:
x.shape

(20000, 20000)

In [ ]:
y = df['label']

In [ ]:
y.shape

(20000,)

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

gnb= GaussianNB()
mul = MultinomialNB()
ber= BernoulliNB()


In [ ]:
gnb.fit(x_train,y_train)
prediction1= gnb.predict(x_test)

print(accuracy_score(y_test,prediction1))
print(confusion_matrix(y_test,prediction1))
print(precision_score(y_test,prediction1))

0.6745
[[1528  440]
 [ 862 1170]]
0.7267080745341615


In [ ]:
mul.fit(x_train,y_train)
prediction2= mul.predict(x_test)

print(accuracy_score(y_test,prediction2))
print(confusion_matrix(y_test,prediction2))
print(precision_score(y_test,prediction2))

0.85075
[[1702  266]
 [ 331 1701]]
0.8647686832740213


In [ ]:
ber.fit(x_train,y_train)
prediction3= ber.predict(x_test)

print(accuracy_score(y_test,prediction3))
print(confusion_matrix(y_test,prediction3))
print(precision_score(y_test,prediction3))

0.84975
[[1675  293]
 [ 308 1724]]
0.8547347545860189


In [ ]:
import pickle

pickle.dump(tf,open("tfidf.pkl","wb"))
pickle.dump(mul,open("model.pkl","wb"))